In [ ]:
!pip install pygame

In [ ]:
import pygame
import random
import pickle
import os

pygame.init()

WHITE = (255, 255, 255)
WIDTH, HEIGHT = 400, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Clumsy Bird Flies Early")

FPS = 30
BIRD_SIZE = 20
PIPE_WIDTH = 50
PIPE_GAP = 150
GRAVITY = 1
FLAP_STRENGTH = -10

ACTIONS = [0, 1] 
ALPHA = 0.2       
GAMMA = 0.9     
EPSILON = 0.2     
EPSILON_DECAY = 0.99
EPSILON_MIN = 0.01

Q_FILE = "q_table.pkl"
if os.path.exists(Q_FILE):
    with open(Q_FILE, "rb") as f:
        Q = pickle.load(f)
else:
    Q = {}

def get_state(bird_y, pipe_x, pipe_y, bird_velocity):
    return (bird_y // 10, pipe_x // 10, pipe_y // 10, int(bird_velocity))

def get_q(state, action):
    return Q.get((state, action), 0)

def set_q(state, action, value):
    Q[(state, action)] = value

def save_q_table():
    with open(Q_FILE, "wb") as f:
        pickle.dump(Q, f)

background_img = pygame.image.load("background.png")
bird_img = pygame.image.load("bird.png")
pipe_top_img = pygame.image.load("pipe_top.png")
pipe_bottom_img = pygame.image.load("pipe_bottom.png")

background_img = pygame.transform.scale(background_img, (WIDTH, HEIGHT))
bird_img = pygame.transform.scale(bird_img, (BIRD_SIZE, BIRD_SIZE))
pipe_top_img = pygame.transform.scale(pipe_top_img, (PIPE_WIDTH, HEIGHT))
pipe_bottom_img = pygame.transform.scale(pipe_bottom_img, (PIPE_WIDTH, HEIGHT))

class Bird:
    def __init__(self):
        self.x = 50
        self.y = HEIGHT // 2
        self.velocity = 0

    def flap(self):
        self.velocity = FLAP_STRENGTH

    def update(self):
        self.velocity += GRAVITY
        self.y += self.velocity

    def draw(self):
        screen.blit(bird_img, (self.x, self.y))

class Pipe:
    def __init__(self, x):
        self.x = x
        self.y = random.randint(100, HEIGHT - PIPE_GAP - 100)

    def update(self):
        self.x -= 5

    def draw(self):
        screen.blit(pipe_top_img, (self.x, self.y - pipe_top_img.get_height()))
        screen.blit(pipe_bottom_img, (self.x, self.y + PIPE_GAP))

    def collides_with(self, bird):
        bird_rect = pygame.Rect(bird.x, bird.y, BIRD_SIZE, BIRD_SIZE)
        top_pipe_rect = pygame.Rect(self.x, self.y - pipe_top_img.get_height(), PIPE_WIDTH, pipe_top_img.get_height())
        bottom_pipe_rect = pygame.Rect(self.x, self.y + PIPE_GAP, PIPE_WIDTH, HEIGHT - (self.y + PIPE_GAP))
        return bird_rect.colliderect(top_pipe_rect) or bird_rect.colliderect(bottom_pipe_rect)

class Background:
    def __init__(self):
        self.x1 = 0
        self.x2 = WIDTH
        self.speed = 2

    def update(self):
        self.x1 -= self.speed
        self.x2 -= self.speed
        if self.x1 + WIDTH < 0:
            self.x1 = self.x2 + WIDTH
        if self.x2 + WIDTH < 0:
            self.x2 = self.x1 + WIDTH

    def draw(self):
        screen.blit(background_img, (self.x1, 0))
        screen.blit(background_img, (self.x2, 0))

def main():
    global EPSILON

    bird = Bird()
    pipes = [Pipe(WIDTH + i * 200) for i in range(3)] 
    background = Background()
    clock = pygame.time.Clock()
    score = 0
    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        pipe = next((p for p in pipes if p.x + PIPE_WIDTH > bird.x), None)
        if not pipe:
            pipe = pipes[0]

        state = get_state(bird.y, pipe.x, pipe.y, bird.velocity)
        if random.random() < EPSILON:
            action = random.choice(ACTIONS)
        else:
            q_values = [get_q(state, a) for a in ACTIONS]
            action = q_values.index(max(q_values))

        if action == 1 or bird.y > pipe.y + PIPE_GAP // 2:
            bird.flap()

        bird.update()
        background.update()
        for pipe in pipes:
            pipe.update()
            if pipe.x + PIPE_WIDTH < 0:
                pipes.remove(pipe)
                new_pipe_x = pipes[-1].x + 200 
                pipes.append(Pipe(new_pipe_x))
                score += 1

        if bird.y < 0 or bird.y > HEIGHT or any(pipe.collides_with(bird) for pipe in pipes):
            save_q_table()
            bird = Bird()
            pipes = [Pipe(WIDTH + i * 200) for i in range(3)]
            score = 0

        next_state = get_state(bird.y, pipe.x, pipe.y, bird.velocity)
        pipe_center = pipe.y + PIPE_GAP // 2
        distance_to_center = abs(bird.y - pipe_center)
        reward = 0.1
        if bird.y < 0 or bird.y > HEIGHT:
            reward = -100
        if any(pipe.collides_with(bird) for pipe in pipes):
            reward = -100
        if bird.y > 0 and bird.y < HEIGHT and pipe.x + PIPE_WIDTH < bird.x:
            reward += 10
        else:
            reward += (1 - (distance_to_center / HEIGHT)) * 0.5

        max_next_q = max(get_q(next_state, a) for a in ACTIONS)
        old_q = get_q(state, action)
        new_q = old_q + ALPHA * (reward + GAMMA * max_next_q - old_q)
        set_q(state, action, new_q)

        if EPSILON > EPSILON_MIN:
            EPSILON *= EPSILON_DECAY

        background.draw()
        bird.draw()
        for pipe in pipes:
            pipe.draw()
        font = pygame.font.Font(None, 36)
        score_text = font.render(f"Score: {score}", True, WHITE)
        screen.blit(score_text, (10, 10))
        pygame.display.flip()
        clock.tick(FPS)

    save_q_table()

if __name__ == "__main__":
    main()
    pygame.quit()